# Work through the reduction of a single dataset

In [1]:
# imports
import os
import glob
import pdb
#from imp import reload
#from importlib import reload

from astropy.io import fits

from cosredux import utils as cr_utils
from cosredux import trace as cr_trace
from cosredux import darks as cr_darks
from cosredux import io as cr_io
from cosredux import science as cr_science

In [ ]:
# python setup.py develop

## Setup files
    Copy files from Dropbox to local, working folder
    cd 'working_folder'
    # Darks, if needed
    cp -rp ~/Dropbox/COS-LRG/darksall .
    # Calibs, as needed
    cp -rp ~/Dropbox/COS-LRG/calibfilesmast .
    # Subset of the raw and object calibration files
    mkdir LCYA01010
    cd LCYA01010
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/*rawtag* .
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/lcya01010_asn.fits .
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/lcya01010_trl.fits .
    cp -rp ~/Dropbox/COS-LRG/LCYA01010/lcya01010_j*.fits .

In [2]:
rdx_path = '/Users/xavier/HST/COS/LRG_Redux/'
#rdx_path = '/home/marijana/ReductionCOS/'
science_folder = 'LCYA01010/'
dark_folder = 'darksall/'
#calib_folder = 'calibfilesmast/'
calib_folder = 'calibs/'
root_out = 'lcya01010'

In [3]:
# Default values
defaults = {}
defaults['pha_mnx'] = (2,15)
defaults['apert'] = 25.

## Customize for calcos
    Science frames
    LP2 calib file

### Science frames

In [3]:
cr_utils.modify_rawtag_for_calcos(rdx_path+science_folder)

Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_rawtag_a.fits


### Calibs -- May not need to repeat this (i.e. if done before)

In [4]:
cr_utils.modify_LP2_1dx_calib(rdx_path+calib_folder)

----

## Run calcos

### Launch pyraf
    cd ureka
    xgterm (or xterm)
    export lref=/home/marijana/ReductionCOS/calibfilesmast/
    # Working in xgterm window
    ur_setup
    mkiraf #?
    pyraf

### Execute
    cd folder_with_data  (e.g. LCYA01010)
    stsdas
    hst_calib
    hstcos
    #
    calcos [dataset prefix]_asn.fits (e.g. calcos lcya01010_asn.fits)

#### Crashing and rerunning
    If you have to rerun, remove the corrtag and .tra files

----

## Coadd corrtag files

In [3]:
corrtag_files_a = glob.glob(rdx_path+science_folder + '*_corrtag_a.fits')
corrtag_files_b = glob.glob(rdx_path+science_folder + '*_corrtag_b.fits')

In [4]:
corrtag_files_a

['/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01fyq_corrtag_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01g0q_corrtag_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01ggq_corrtag_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01gjq_corrtag_a.fits']

In [24]:
_ = cr_utils.coadd_bintables(corrtag_files_a, outfile=rdx_path+science_folder+root_out+'_coaddcorr_woPHA_a.fits')
_ = cr_utils.coadd_bintables(corrtag_files_b, outfile=rdx_path+science_folder+root_out+'_coaddcorr_woPHA_b.fits')

Wrote outfile /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01010_coaddcorr_woPHA_a.fits
Wrote outfile /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01010_coaddcorr_woPHA_b.fits


## Traces

In [25]:
reload(cr_trace)
traces_a=cr_trace.traces(rdx_path+science_folder+root_out+'_coaddcorr_woPHA_a.fits', rdx_path+calib_folder, 'FUVA', clobber=True)
traces_b=cr_trace.traces(rdx_path+science_folder+root_out+'_coaddcorr_woPHA_b.fits', rdx_path+calib_folder, 'FUVB', clobber=True)

Updated trace for segment=FUVA in /Users/xavier/HST/COS/LRG_Redux/calibs/x6q17586l_1dx.fits
Wrote Traces to {:s} /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01010_coaddcorr_woPHA_a_traces.json
Updated trace for segment=FUVB in /Users/xavier/HST/COS/LRG_Redux/calibs/x6q17586l_1dx.fits
Wrote Traces to {:s} /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01010_coaddcorr_woPHA_b_traces.json


## Ignoring Sun and Limb

## Get HVLEVELs

In [18]:
reload(cr_utils)
hva_a, hvb_a = cr_utils.get_hvlevels(corrtag_files_a)
hva_b, hvb_b = cr_utils.get_hvlevels(corrtag_files_b)

HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_corrtag_a.fits
HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_corrtag_a.fits
HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_corrtag_a.fits
HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_corrtag_a.fits
HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_corrtag_b.fits
HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_corrtag_b.fits
HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_corrtag_b.fits
HVA=167 and HVB=169 in file /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_corrtag_b.fits


## Change PHA

#### Examine PHA values near the trace of a single exposure  (OPTIONAL)

In [91]:
reload(cr_science)
ex_region = cr_science.set_extraction_region(traces_a[0], 'FUVA', corrtag_files_a[0], check=True)

In [94]:
reload(cr_darks)
pha_values_a, _, _ = cr_darks.get_pha_values_science(ex_region, corrtag_files_a[0], background=False)

In [95]:
from xastropy.xutils import xdebug as xdb
xdb.xhist(pha_values_a)

#### Edit

In [97]:
# Reset pha_mnx above if you wish

In [23]:
reload(cr_utils)
cr_utils.change_pha(rdx_path+calib_folder, low=defaults['pha_mnx'][0], up=defaults['pha_mnx'][1])

Updating PHA values in /home/marijana/ReductionCOS/calibfilesmast/wc318317l_pha.fits


## Modify PHACORR

In [25]:
reload(cr_utils)
cr_utils.modify_phacorr(rdx_path+science_folder)

Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01ggq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01fyq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01gjq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/LCYA01010/lcya01g0q_rawtag_a.fits


## Clean for CALCOS

In [9]:
reload(cr_utils)
cr_utils.clean_for_calcos_phafiltering(rdx_path+science_folder)

## Run Calcos as above -- with PHA restricted

### Coadd new PHA frames

In [21]:
corrtag_files_a = glob.glob(rdx_path+science_folder + '*_corrtag_a.fits')
corrtag_files_b = glob.glob(rdx_path+science_folder + '*_corrtag_b.fits')

In [23]:
corrtag_files_a

['/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01fyq_corrtag_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01g0q_corrtag_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01ggq_corrtag_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01gjq_corrtag_a.fits']

In [24]:
_ = cr_utils.coadd_bintables(corrtag_files_a, outfile=rdx_path+science_folder+root_out+'_coaddcorr_withPHA_a.fits')
_ = cr_utils.coadd_bintables(corrtag_files_b, outfile=rdx_path+science_folder+root_out+'_coaddcorr_withPHA_b.fits')

Wrote outfile /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01010_coaddcorr_withPHA_a.fits
Wrote outfile /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01010_coaddcorr_withPHA_b.fits


## Reduce Darks 

### Find Darks

In [31]:
darks_a=cr_utils.find_darks(rdx_path+dark_folder, corrtag_files_a[0], 'a')
darks_b=cr_utils.find_darks(rdx_path+dark_folder, corrtag_files_b[0], 'b')

In [32]:
darks_a[0:3]

['/home/marijana/ReductionCOS/darksall/ld0x3yn6q_rawtag_a.fits',
 '/home/marijana/ReductionCOS/darksall/ld0x4vjhq_rawtag_a.fits',
 '/home/marijana/ReductionCOS/darksall/ld0x5ebhq_rawtag_a.fits']

### Copy Darks into sub-folders

### Edit headers
    Do not need to repeat if done previously

In [33]:
cr_utils.modify_rawtag_for_calcos(rdx_path+dark_folder)

Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/ld0x3yn6q_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/lcr96wwvq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/ld0x3rc4q_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/ld0x09noq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/ld0x12yqq_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/ld0x35glq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/lcr96luxq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/lcr9b5k2q_rawtag_b.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/ld0x4vjhq_rawtag_a.fits
Modifying header cards for rawtag file: /home/marijana/ReductionCOS/darksall/ld0x0kbuq_rawt

### Generate darks script

### Find voltages that darks have 

## Measure Background

### Set background region
    Set chk=True to show plots
    Iterate as needed

In [16]:
# Read traces (if needed)
traces_a = cr_io.read_traces(rdx_path+science_folder+root_out+'_coaddcorr_woPHA_a.fits')
traces_b = cr_io.read_traces(rdx_path+science_folder+root_out+'_coaddcorr_woPHA_b.fits')

In [17]:
reload(cr_darks)
chk = True
bg_region_a = cr_darks.set_background_region(traces_a[0], 'FUVA', rdx_path+science_folder+root_out+'_coaddcorr_woPHA_a.fits', check=chk)
bg_region_b = cr_darks.set_background_region(traces_b[0], 'FUVB', rdx_path+science_folder+root_out+'_coaddcorr_woPHA_b.fits', check=chk)
bg_region_a, bg_region_b

({u'lower': (1200.0, 2445.5107, 431.1351318359375, 381.1351318359375),
  u'upper': (1200.0, 2445.5107, 491.1351318359375, 541.1351318359375)},
 {u'lower': (50.0, 1209.924, 490.27716064453125, 440.27716064453125)})

### Loop on Exposures without PHA
    Will generate background spectra, one per exposure per segment
    Below we only do FUVA

In [18]:
corrtag_woPHA_a = glob.glob(rdx_path+science_folder + '*_corrtag_woPHA_a.fits')
corrtag_woPHA_a

['/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01fyq_corrtag_woPHA_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01g0q_corrtag_woPHA_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01ggq_corrtag_woPHA_a.fits',
 '/Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01gjq_corrtag_woPHA_a.fits']

In [77]:
reload(cr_darks)
cr_darks.dark_to_exposures(corrtag_woPHA_a, bg_region_a, traces_a[0], 'FUVA', defaults)

HVA=167 and HVB=169 in file /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01fyq_corrtag_woPHA_a.fits
HVA=167 and HVB=169 in file /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01g0q_corrtag_woPHA_a.fits
HVA=167 and HVB=169 in file /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01ggq_corrtag_woPHA_a.fits
HVA=167 and HVB=169 in file /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01gjq_corrtag_woPHA_a.fits
Working on exposure: /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01fyq_corrtag_woPHA_a.fits
Matching darks to the exposure
We have 14 darks to use
Wrote outfile /Users/xavier/HST/COS/LRG_Redux/LCYA01010/darks_a/a_167/lcya01fyq_darks.fits
Will use scale factor = 0.079273
Extracting..
Wrote background spectrum to /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01fyq_a_bkgd.fits
Working on exposure: /Users/xavier/HST/COS/LRG_Redux/LCYA01010/lcya01g0q_corrtag_woPHA_a.fits
Matching darks to the exposure
We have 25 darks to use
Wrote outfile /Users/xavier/HST/COS/LRG_Redux/LCYA01010/darks_a/a_16

### Coadd

In [69]:
x1d_files = glob.glob(rdx_path+science_folder + '*_x1d.fits')

In [81]:
reload(cr_science)
cr_science.coadd_exposures(x1d_files, 'FUVA', rdx_path+science_folder+'LCYA01010_coadd.fits')

Wrote /Users/xavier/HST/COS/LRG_Redux/LCYA01010/LCYA01010_coadd.fits
